# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import sklearn.datasets as sk
iris = sk.load_iris()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

In [ ]:
df= pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                 columns= iris['feature_names'] + ['target'])

df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
train_df = pd.DataFrame(data= np.c_[X_train, y_train], columns= iris['feature_names'] + ['species'])
test_df = pd.DataFrame(data= np.c_[X_test, y_test], columns= iris['feature_names'] + ['species'])

In [ ]:
leaf_count = 0
from sklearn.metrics import accuracy_score, mean_squared_error
class Node:
    def __init__(self, t, depth, feature, side):
        self.t = t
        self.depth = depth
        self.feature = feature
        self.leftChild = None
        self.rightChild = None
        self.side = side
        self.target = None
    def setTarget(self, target):
        self.target = target
    def predict(self, cur_el):
        value = self.target
        if (self.leftChild != None and self.rightChild != None):
            if (float(cur_el[self.feature].to_string(index=False)) < self.t):
                value = self.leftChild.predict(cur_el)
            else:
                value = self.rightChild.predict(cur_el)
        return value
    def print(self):
        print(self.t, ", depth: ", self.depth, ", feature: ", self.feature, ", side: ", self.side)
        if self.leftChild != None:
            self.leftChild.print()
        if self.rightChild != None:
            self.rightChild.print()

def calc_Q(df):
    #pick best feature:
    features = ['sepal length (cm)', 'petal length (cm)', 'sepal width (cm)', 'petal width (cm)']
    min_id = -1
    for i in range(len(features)):
      grid = np.linspace(df[features[i]].min(), df[features[i]].max(), 10)
      best_t = grid[0]
      best_IG = -1
      for t in grid:
          sample1 = df[df[features[i]] < t]
          sample2 = df[df[features[i]] >= t]
          IG = entropy(df['species']) - ((len(sample1['species'])/df['species'].shape[0]) * entropy(sample1['species']) + (len(sample2['species'])/df['species'].shape[0]) * entropy(sample2['species']))
          if IG > best_IG:
              best_t, best_IG, min_id  = t, IG, i
    return best_t, features[min_id]
    

def split_node(depth, df, side):
    # global leaf_count
    if (entropy(df['species']) == 0 or depth > 4 or df.shape[0] < 6): # Stop critereas
        # leaf node
        leaf_node = Node(None, depth, None, side + "leaf")
        if len(df['species']) != 0:
           leaf_node.setTarget(float(df.iloc[[0]]['species'].to_string(index=False)))
        # leaf_count += 1
        # print("Leaf_count: ", leaf_count)
        return leaf_node
    # here we calc Q
    t, feature = calc_Q(df)
    node = Node(t, depth, feature, side)
    Rleft = df[df[feature] < t]
    Rright = df[df[feature] >= t]
    node.leftChild = split_node(depth+1, Rleft, "left")
    node.rightChild = split_node(depth+1, Rright, "right")
    return node

def predict(test_df, start_node):
    predicted = []
    for i in range(len(test_df)):
        cur_el = test_df.iloc[[i]]
        predicted.append(start_node.predict(cur_el))
    return predicted
        

#first we train, we train, using train_df
start_node = split_node(1, train_df, "root")
predicted = predict(test_df, start_node)

print("Predicted: ", " len of predicted: ", len(predicted), ", predicted itself: ", predicted)
print("Real result: ", " len of result: ", len(y_test), ", real itself: ",y_test)
# start_node.print()
print("accuracy_test: ", accuracy_score(y_test, predicted))

from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier(max_depth=4, criterion='entropy').fit(X_train, y_train).score(X_test, y_test)




Predicted:   len of predicted:  50 , predicted itself:  [1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 1.0, 1.0, 2.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 2.0, 1.0, 2.0]
Real result:   len of result:  50 , real itself:  [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2]
accuracy_test:  0.98


0.98

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
%%capture
!wget https://www.dropbox.com/s/zvx8zy0khjaijt5/churn.csv


In [ ]:
df = pd.read_csv('churn.csv')
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], 1)
df['Geography'] = df.groupby("Geography")["Exited"].transform("mean")
df['Gender'] = df.groupby("Gender")["Exited"].transform("mean")
# df = df.drop('')

In [ ]:
print(list(df.columns.values))

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:10], df['Exited'], test_size=0.33, random_state=random.randint(1,100))

In [ ]:
train_df = pd.DataFrame(data= np.c_[X_train, y_train], columns=df.columns.values[:])

In [ ]:
print(train_df)

      CreditScore  Geography    Gender  ...  IsActiveMember  EstimatedSalary  Exited
0           715.0   0.161548  0.250715  ...             1.0        181600.72     0.0
1           616.0   0.166734  0.164559  ...             1.0        173599.38     0.0
2           687.0   0.161548  0.250715  ...             1.0         47921.22     0.0
3           721.0   0.161548  0.164559  ...             1.0          4493.12     0.0
4           655.0   0.161548  0.164559  ...             1.0        142415.97     0.0
...           ...        ...       ...  ...             ...              ...     ...
6695        695.0   0.166734  0.250715  ...             1.0        127977.66     0.0
6696        594.0   0.161548  0.164559  ...             0.0        186884.04     0.0
6697        598.0   0.324432  0.164559  ...             1.0        136886.86     0.0
6698        603.0   0.324432  0.164559  ...             0.0        152541.89     1.0
6699        492.0   0.324432  0.250715  ...             0.0      

In [ ]:
class RandomForest:
    def __init__(self):
        self.trees = None
        self.predicted = None
        self.finalvars = None
        self.X_train = None
        self.y_train = None
        
    def get_random_sample(self, train_df):
        lol = train_df.sample(frac=random.random(),random_state=34)
        return lol.iloc[:, 0:10], lol['Exited']
    def fit(self, train_df, amount_of_trees=100, max_depth=None, max_leaf_nodes=None, min_samples_leaf=1, criterion='entropy', max_features='sqrt'):
        self.trees = []
        for i in range(amount_of_trees):
            newtree = DecisionTreeClassifier(max_depth=max_depth, criterion=criterion, max_leaf_nodes=max_leaf_nodes, min_samples_leaf=min_samples_leaf, max_features=max_features)
            self.X_train, self.y_train = self.get_random_sample(train_df)
            newtree.fit(self.X_train, self.y_train)
            self.trees.append(newtree)
    def get_most_important_features(self):
        feature_importance = np.zeros(10)
        for i in range(len(self.trees)):
            feature_importance = np.add(feature_importance, self.trees[i].feature_importances_)
        feature_importance = feature_importance / len(self.trees)
        return zip(X_train.columns.values, feature_importance)
    def predict(self, X_test):
        self.predicted = np.zeros(len(X_test))
        print(len(self.trees))
        for i in range(len(self.trees)):
            self.predicted = self.predicted + self.trees[i].predict(X_test)
        self.finalvars = self.predicted / len(self.trees)
        ans = random.choice(self.trees)
        return np.around(self.finalvars)
        # return ans.predict(X_test)
newforest = RandomForest()
newforest.fit(train_df, 20)
predicted = newforest.predict(X_test)
print(accuracy_score(y_test, predicted))
print(calc_acc(y_test, predicted))


      


20
0.8563636363636363
test: [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,

In [ ]:
features = list(newforest.get_most_important_features())
features = sorted(features, key=lambda x: x[1], reverse=True)
print(features)

[('Age', 0.23831408118183411), ('EstimatedSalary', 0.1500775611665707), ('CreditScore', 0.1494643877748199), ('Balance', 0.1372866715060259), ('NumOfProducts', 0.1239687757660918), ('Tenure', 0.08586759839183741), ('Geography', 0.04258898272760827), ('IsActiveMember', 0.03458784079801133), ('Gender', 0.019792531222092048), ('HasCrCard', 0.01805156946510863)]


Вывод: самыми важными признаками являются: Age, EstimatedSalary, CreditScore, Balance;

In [ ]:
test_cases = {'amount_of_trees': [1, 10, 100, 1000], 'max_depth':[None, 10, 100, 1000], 'max_leaf_nodes':[None, 10, 100, 1000], 
              'min_samples_leaf':[1, 5, 40, 80], 'criterion': ['entropy', 'gini'], 'max_features':['auto', 'sqrt', 'log2', 4, 9]}
for i in range(10):
    amount_of_trees = random.choice(test_cases['amount_of_trees'])
    max_depth = random.choice(test_cases['max_depth'])
    max_leaf_nodes = random.choice(test_cases['max_leaf_nodes'])
    min_samples_leaf = random.choice(test_cases['min_samples_leaf'])
    criterion = random.choice(test_cases['criterion'])
    max_features = random.choice(test_cases['max_features'])
    forest = RandomForest()
    forest.fit(train_df, amount_of_trees, max_depth, max_leaf_nodes, min_samples_leaf, criterion, max_features)
    print("Params of cur iter are: ", amount_of_trees, ", ", max_depth, ", ", max_leaf_nodes, ", ", min_samples_leaf, ", ", criterion, ", ", max_features,", acc: ", accuracy_score(y_test, forest.predict(X_test)))
    del forest

10
Params of cur iter are:  10 ,  1000 ,  10 ,  80 ,  entropy ,  9 , acc:  0.823030303030303
1000
Params of cur iter are:  1000 ,  1000 ,  1000 ,  5 ,  gini ,  4 , acc:  0.8615151515151516
10
Params of cur iter are:  10 ,  100 ,  None ,  5 ,  gini ,  9 , acc:  0.8475757575757575
1000
Params of cur iter are:  1000 ,  100 ,  100 ,  1 ,  gini ,  9 , acc:  0.8572727272727273
1
Params of cur iter are:  1 ,  None ,  10 ,  40 ,  gini ,  4 , acc:  0.8351515151515152
10
Params of cur iter are:  10 ,  None ,  10 ,  80 ,  entropy ,  sqrt , acc:  0.8106060606060606
1000
Params of cur iter are:  1000 ,  10 ,  100 ,  40 ,  gini ,  9 , acc:  0.8436363636363636
1
Params of cur iter are:  1 ,  1000 ,  100 ,  80 ,  gini ,  sqrt , acc:  0.8121212121212121
100
Params of cur iter are:  100 ,  100 ,  10 ,  80 ,  gini ,  9 , acc:  0.833939393939394
10
Params of cur iter are:  10 ,  100 ,  10 ,  5 ,  entropy ,  auto , acc:  0.8318181818181818


Один из наилучших результатов дала модель со следующими параметрами: 1000 ,  1000 ,  1000 ,  5 ,  gini ,  4 , acc:  0.8615151515151516